In [3]:
import argparse
import json
from torch.utils.data import DataLoader
from thop import profile
from ptflops import get_model_complexity_info

In [4]:
from models import *
from utils.datasets import *
from utils.utils import *
from quant_dorefa import *

from mymodel import *
from torchsummary import summary

In [5]:
model = torch.load('weights/4w4a/test_best.pt', map_location=torch.device('cpu'))
# model = UltraNet()
# model.load_state_dict(checkpoint[])
# model.hyp = hyp
# model.nc = 1
# model.arc = 'default'

In [7]:
# print(model.parameters())
# print(model['model'])
# for layers in model:
#     print(layers)
# summary(model, (3, 320, 160))

In [8]:
print("Model's state_dict:")
for param_tensor in model['model']:
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    if(len(model['model'][param_tensor].size()) > 1):
        print(param_tensor, model['model'][param_tensor].size())

# print(model['model']['layers.0.weight'])    

Model's state_dict:
layers.0.weight torch.Size([16, 3, 3, 3])
layers.4.weight torch.Size([32, 16, 3, 3])
layers.8.weight torch.Size([64, 32, 3, 3])
layers.12.weight torch.Size([64, 64, 3, 3])
layers.16.weight torch.Size([64, 64, 3, 3])
layers.19.weight torch.Size([64, 64, 3, 3])
layers.22.weight torch.Size([64, 64, 3, 3])
layers.25.weight torch.Size([64, 64, 3, 3])
layers.28.weight torch.Size([36, 64, 1, 1])


In [9]:
conv_0_weight = model['model']['layers.0.weight']
conv_1_weight = model['model']['layers.4.weight']
conv_2_weight = model['model']['layers.8.weight']
conv_3_weight = model['model']['layers.12.weight']
conv_4_weight = model['model']['layers.16.weight']
conv_5_weight = model['model']['layers.19.weight']
conv_6_weight = model['model']['layers.22.weight']
conv_7_weight = model['model']['layers.25.weight']
conv_8_weight = model['model']['layers.28.weight']
# print(conv_0_weight.data[0])
# torch.save(conv_0_weight, 'conv_0_original.txt')

In [10]:
def get_w_quant(in_f, k):
    in_f = torch.tanh(in_f)
    n = float(2 ** (k-1)  - 1)
    out = torch.round(in_f * n) / n
    return out

def get_w_int(in_f, k):
    in_f = torch.tanh(in_f)
    n = float(2 ** (k-1)  - 1)
    out = torch.round(in_f * n)
    return out

In [11]:
def print_hex_line(in_f, size_value, SIMD, PE):
    print(size_value)
    print('{', end='')
    for i in range(0, size_value[0]):  # OUT_channel traverse
        if i%(size_value[0]/PE) == 0:
            print('')
            print('{', end = '')
        for k1 in range(0, size_value[2]): # kernel trverse
            for k2 in range(0, size_value[3]): # kernel traverse
                for j in range(0, size_value[1]): # In_channel traverse
                    if in_f[i][j][k1][k2] < 0:
                        in_f[i][j][k1][k2] = in_f[i][j][k1][k2] * (-1) + 8
                    if j/(size_value[1]/SIMD) == 0:
                        print('"0x', end='')
                    formatted_data = format(int(in_f[i][j][k1][k2].item()), "x")
#                     print(formatted_data, end = '')
                    print(i, end='')
                    if size_value[1] <= SIMD:
                        if i == size_value[0]/PE -1:
                            print('"}', end='')
                        else:
                            if j == size_value[1]-1:
                                print('",', end='')
#                             else:
                    else:
                        if int((j+1)%(size_value[1]/SIMD)) == 0:
                            print('",', end='')
    print('}', end = '')

In [12]:
def weight_reorg_2d(in_f, size_value, SIMD, PE):
    print(size_value)
    K = size_value[2]
    IN_CH = size_value[1]
    OUT_CH = size_value[0]
#     weight_store[OUT_CH][OUT_CH*IN_CH*K*K]
    rows, cols = (OUT_CH, IN_CH*K*K)
    weight_store = [[0 for i in range(cols)] for j in range(rows)]
#     dim_1 = PE
#     dim_2 = K*K*OUT_CH/PE*IN_CH*SIMD
    
    for out_channel in range(0, OUT_CH):
        for in_channel in range(0, IN_CH):
            for ker1 in range(0, K):
                for ker2 in range(0, K):
                    weight_store[out_channel][in_channel*K*K + ker1*K + ker2] = int(in_f[out_channel][in_channel][ker1][ker2].item());
    return weight_store, rows, cols

def weight_print(weight_2d, rows, cols, SIMD, PE):
    print('out_dim = ', int(PE), ',', int((cols/SIMD)*(rows/PE)))
    print('{')
    for i in range(0, rows):
        if i%(rows/PE) == 0:
            print('')
            print('{')
        simd_cnt = 0
        for j in range(0, cols):
            if weight_2d[i][j] < 0:
                weight_2d[i][j] = weight_2d[i][j] * (-1) + 8
            if j%SIMD == 0:
                print('"0x', end='')
            formatted_data = format(int(weight_2d[i][j]), "x")
            print(formatted_data, end='')
            simd_cnt = simd_cnt + 1
            if simd_cnt%SIMD == 0:
                if j == cols-1 and (i+1)%(rows/PE)==0:
                    if j == cols -1 and i == rows-1:
                        print('"}')
                    else:
                        print('"},', end='')
                else:
                    print('",', end='')
    print('}')

In [18]:
conv_0_wq = get_w_int(conv_8_weight, 4)
PE = 2
SIMD = 8
# print(conv_1_wq.data[0][0])
size_value = conv_0_wq.size()
# conv_0_wq = conv_0_wq.cpu()
# conv_0_wq = get_w_int(conv_0_wq, 4)
weight_store, rows, cols = weight_reorg_2d(conv_0_wq, size_value, 0, 0)
print(rows, cols)
weight_print(weight_store, rows, cols, SIMD, PE)

torch.Size([36, 64, 1, 1])
36 64
out_dim =  2 , 144
{

{
"0x910e00a0","0x606e0000","0x66040001","0xee0e006e","0x00001609","0xb0300000","0x01000ee6","0x500e0010","0xa602e03a","0x50600000","0x0ee6200d","0x0a9001b6","0x09e06060","0x60e00190","0xe200e6e0","0xd00de606","0x31010c60","0x00003010","0x01000daa","0x00190019","0x02000095","0x000004e0","0x04001099","0x0a000030","0xa1000030","0x00102020","0x091109ab","0x00c00001","0x040000a0","0x1090b0a0","0x01004090","0x0d00009d","0x4ddeeb4e","0xde6eaebe","0xe21ddddc","0xe6c0eed4","0xeceeee1b","0xbe3eba9e","0xdaee0d04","0x9ceddeca","0x01900000","0x90011000","0x19000001","0x99101099","0x91090001","0x00090900","0x00000090","0x00019000","0x010e00a0","0x606e0000","0x66040001","0xee0e006e","0x00001609","0xb0300000","0x01000ee6","0x500e0010","0xa602e03a","0x50600000","0x0ee6200d","0x0a9000b6","0x09e06060","0x60e00190","0xe200e6e0","0xd00de606","0x31010c60","0x00003010","0x01000daa","0x00190019","0x02000095","0x000004e0","0x04001099","0x0a000030","0xa100

In [ ]:
conv_0_wq = get_w_int(conv_0_weight, 4)
PE = 2
SIMD = 8
# print(conv_1_wq.data[0][0])
size_value = conv_0_wq.size()
conv_0_wq = conv_0_wq.cpu()
print_hex_line(conv_0_wq.data, size_value, SIMD, PE)
# print(size_value[1])

In [ ]:
print(size_value)